# p5.js in the Jupyter Notebook for custom interactive visualizations 

## by Jeremy Tuloup

- [@jtpio](https://twitter.com/jtpio)
- [jtp.io](https://jtp.io)

#### PyData Berlin Meetup - February 2018

##  The Python Visualization Landscape

![Python Landscape](./img/python_viz_landscape.png)

## What is p5.js?

- JS client-side library for creating graphic and interactive experiences
- Based on the core principles of Processing
- More infos on the website: [https://p5js.org](https://p5js.org)

![P5 Editor](./img/p5.png)

[http://alpha.editor.p5js.org/p5/sketches/BygPr3zaZYW](http://alpha.editor.p5js.org/p5/sketches/BygPr3zaZYW)

## How to execute Javascript code in the Jupyter Notebook?


- Use the `%%javascript` cell magic

In [ ]:
%%javascript

// type your Javascript code here

let myVar = 'test'
element.text(myVar)

## How to use p5.js in the Jupyter Notebook?

- Using require.js
- p5.js: rendering on the canvas
- p5.dom: creating dom elements
- lodash: utility library with many high order functions

In [ ]:
%%javascript
require.config({
    paths: {
        'p5': 'https://cdnjs.cloudflare.com/ajax/libs/p5.js/0.6.0/p5.min',
        'p5.dom': 'https://cdnjs.cloudflare.com/ajax/libs/p5.js/0.6.0/addons/p5.dom.min',
        'lodash': 'https://cdnjs.cloudflare.com/ajax/libs/lodash.js/4.17.4/lodash.min'
    }
});

In [ ]:
%%javascript

window.defineModule = function (name, dependencies, module) {
    // force the recreation of the module
    // (when re-executing a cell for example)
    require.undef(name);
    
    define(name, dependencies, module);
};

In [ ]:
%%javascript

window.createSketch = function (cell, name, dependencies, module) {
    // global store for sketches, lazily created
    window.sketches = window.sketches || {};
    
    // remove existing sketch if it already exists
    $(`${name}`).remove();
    cell.append(`<div id="${name}" style="text-align: center;"></div>`);
    
    // stop the existing sketch
    let sketch = window.sketches[name];
    if (sketch) {
        sketch.remove();
    }
    
    require(['p5', 'p5.dom'], (p5, p5dom) => {
        require(dependencies, function(...deps) {
            let sketch = module(...deps);
            window.sketches[name] = new p5(sketch, name);
            console.log(`sketch ${name} created`);
        })
    });
}

In [ ]:
%%javascript

defineModule('testModule', [], () => {
    const W = 500;
    const H = 500;
    
    return {W, H};
})

In [ ]:
%%javascript

createSketch(element, 'test-sketch', ['testModule'], function (testModule) {
    return function(p) {
        const {W, H} = testModule;
        const [CX, CY] = [W / 2, H / 2];
        
        p.setup = function(){
            p.createCanvas(W, H);
            p.rectMode(p.CENTER);
        }

        p.draw = function () {
            p.background('#ddd');
            p.translate(CX, CY);
            p.fill(255, 0, 0);
            p.rect(0, 0, 200, 200); 
        }
    };
});

In [ ]:
%%javascript

createSketch(element, 'test-sketch-3d', ['testModule', 'lodash'], function (testModule, _) {
    return function(p) {
        const {W, H} = testModule;
        
        p.setup = function(){
            p.createCanvas(W, H, p.WEBGL);
        }

        p.draw = function () {
            p.background('#ddd');
            let t = p.frameCount;
            let n = 10;
            p.randomSeed(42);
            _.range(n).forEach(i => {
                const R = 180// + 30 * p.sin(t * 0.2 + i);
                const x = R * p.cos(i * p.TWO_PI / n);
                const y = R * p.sin(i* p.TWO_PI / n);
                p.push();
                p.translate(x, y);
                p.fill(p.random(255), p.random(255), p.random(255));
                p.rotateY(t * 0.05 + i);
                p.box(50);
                p.pop();
            });
        }
    };
});

## Example with the Bermuda Triangle Puzzle

![Bermuda Triangle Puzzle](./img/bermuda_triangle_puzzle.jpg)

## How to solve it?

1. Play with it
2. Manual brute-force
3. Brute-force program
4. Stochastic search. Refer to [Aristotle Number Puzzle](https://jtp.io/2017/01/12/aristotle-number-puzzle.html) and the [Nine-Color Cube](https://jtp.io/2017/05/10/nine-color-cube.html) for more details.

## Strategy

1. Model the puzzle
2. Draw a static state
3. Animate the state transitions
4. Solve the problem and log the steps

## Model

Back to Python!

In [ ]:
N_TRIANGLES = 16
IDS = list(range(N_TRIANGLES))
N_COLORS = 6
WHITE, BLUE, YELLOW, GREEN, BLACK, RED = range(N_COLORS)

# list the pieces, turning anti-clockwise for the colors
# The number at the last position of the tuple indicate the number
# of identical pieces (cf photo above)
triangles_count = [
    (WHITE, BLUE, BLUE, 1),
    (WHITE, YELLOW, GREEN, 2),  # 2 of these
    (WHITE, BLACK, BLUE, 2),  # 2 of these
    (WHITE, GREEN, RED, 1),
    (WHITE, RED, YELLOW, 1),
    (WHITE, WHITE, BLUE, 1),
    (BLACK, GREEN, RED, 1),
    (BLACK, RED, GREEN, 2),  # 2 of these
    (BLACK, BLACK, GREEN, 1),
    (BLACK, GREEN, YELLOW, 1),
    (BLACK, YELLOW, BLUE, 1),
    (GREEN, RED, YELLOW, 1),
    (BLUE, GREEN, YELLOW, 1)
]

assert N_TRIANGLES == sum(t[-1] for t in triangles_count)

In [ ]:
triangles = tuple([t[:-1] for t in triangles_count for times in range(t[-1])])
print(triangles)

assert N_TRIANGLES == len(triangles)

### Board

In [ ]:
import json

class Board:
    TRIANGLES = triangles
    LEFT = (WHITE, RED, WHITE, YELLOW)
    RIGHT = (BLUE, RED, GREEN, BLACK)
    BOTTOM = (GREEN, GREEN, WHITE, GREEN)
    
    def __init__(self, permutation, states=[]):
        self.permutation = list(permutation)
        self.repr = [triangles[p[0]] + (p[1],) for p in self.permutation]
        self.states = states
        
    def __str__(self):
        return json.dumps({
            'TRIANGLES': self.TRIANGLES,
            'LEFT': self.LEFT,
            'RIGHT': self.RIGHT,
            'BOTTOM': self.BOTTOM,
            'repr': self.repr,
            'permutation': self.permutation,
            'states': self.states,
        })
    
    def __repr__(self):
        return f'{self.__class__.__name__}({self.permutation})'

In [ ]:
%%javascript
window.kv = {};

In [ ]:
from IPython.display import Javascript

def send_to_js(key, value):
    display(Javascript(f'window.kv.{key} = {value};'))

In [ ]:
send_to_js('defaultBoard', Board([i, 0] for i in range(N_TRIANGLES)))

## Drawing the state

In [ ]:
%%javascript

// define a list of constant such as the size of the base canvas,
// the size of the triangles, colors...
defineModule('settings', [], () => {
    const ANIM_W = 800;
    const ANIM_H = ANIM_W / 1.6;
    const N_TRIANGLES = 16;
    const COLORS = ['#fff', '#00f', '#ff0', '#0f0', '#000', '#f00'];
    const WOOD_COLOR = '#825201';
    const R = 50;
    const r = R / 2;
    const CR = r / 2;
    const OFFSET_X = R * Math.sqrt(3) * 0.5;
    const OFFSET_Y = 1.5 * R;
    
    return {ANIM_W, ANIM_H, N_TRIANGLES, WOOD_COLOR,
            COLORS, R, r, CR, OFFSET_X, OFFSET_Y};
})

In [ ]:
%%javascript

defineModule('triangles', ['settings'], (settings) => {
    const {COLORS, WOOD_COLOR, R, r, CR, OFFSET_X, OFFSET_Y} = settings;
    
    function _getPoints(n, startAngle, radius) {
        let points = [];
        const da = 2 * Math.PI / n;
        for (let i = 0; i < n; i++) {
            const angle = startAngle - i * da;
            const x = radius * Math.cos(angle);
            const y = radius * Math.sin(angle);
            points.push(x);
            points.push(y);
        }
        return points;
    }
    
    return (p) => {
        return {
            getTrianglePoints: _getPoints,
            genTrianglesPos: function () {
                let positions = [];
                let id = 0;
                for (let row = 0; row < 4; row++) {
                    const n_row = 2 * row + 1;
                    for (let col = 0; col < n_row; col++) {
                        const x = (col - row) * OFFSET_X;
                        const flip = (id + row) % 2;
                        const y = row * OFFSET_Y + ((flip == 1) ? -R/2 : 0);
                        positions.push({id, x, y, flip, row, col, n_row});
                        id++;
                    }
                }
                return positions;
            },
            drawTriangle: function (colors, x, y, rotation, flip=0) {
                const n = colors.length;
                
                p.fill(WOOD_COLOR);
                p.push();
                p.translate(x, y);
                p.rotate(-rotation * p.TWO_PI / 3 + flip * p.PI);
                
                p.triangle(..._getPoints(n, Math.PI / 6, R));

                let circles = _getPoints(n, Math.PI / 2, 1.25 * CR);
                for (let i = 0; i < n; i++) {
                    const xx = circles[2*i];
                    const yy = circles[2*i+1];
                    const color = COLORS[colors[i]];
                    p.fill(color);
                    p.ellipse(xx, yy, CR);
                }
                p.pop();
            }
        };
    };
});

In [ ]:
%%javascript

createSketch(element, 'oneTriangle', ['triangles'], function (Triangles) {
        const [W, H] = [400, 400];

        return function(p) {
            let triangles = Triangles(p);

            p.setup = function(){
                p.createCanvas(W, H);
                p.noLoop();
            }

            p.draw = function () {
                p.background('#ddd');
                p.push();
                triangles.drawTriangle([0, 1, 2], W / 2, H / 2, 0, 0);
                p.pop();
            }
        };
    });

### Static Board

In [ ]:
%%javascript

defineModule('staticBoard', ['settings', 'triangles'], (Settings, Triangles) => {
    let {COLORS, R, CR, OFFSET_X, OFFSET_Y} = Settings;
    
    return (p) => {
        let triangles = Triangles(p);
        
        function _drawStaticColors (board) {
            for (let {id, x, y, row, col, n_row} of triangles.genTrianglesPos()) {
                if (col === 0) {
                    const left = COLORS[board.LEFT[row]];
                    const right = COLORS[board.RIGHT[row]];
                    p.fill(left);
                    p.ellipse(x - OFFSET_X, y - R / 2, CR);
                    p.fill(right);
                    p.ellipse(x + n_row * OFFSET_X, y - R / 2, CR);
                }
                          
                if (row === 3 && col % 2 == 0) {
                    p.fill(COLORS[board.BOTTOM[parseInt(col / 2, 10)]]);
                    p.ellipse(x, 3.75 * OFFSET_Y, CR);
                }
            }
        }
        
        function _drawFrame () {
            const positions = triangles.genTrianglesPos();
            const mid = positions[6];
            p.push();
            p.noFill();
            p.stroke(0);
            p.strokeWeight(2);
            p.translate(mid.x, mid.y);
            p.triangle(...triangles.getTrianglePoints(3, Math.PI / 6, 4 * R));
            p.pop();
        }
        
        return {
            drawStaticColors: _drawStaticColors,
            drawFrame: _drawFrame,
            drawBoard: (boardName) => {
                const board = window.kv[boardName];
                const repr = board.repr;
                
                _drawFrame();
                for (let {id, x, y, flip} of triangles.genTrianglesPos()) {
                    let [a, b, c, rot] = repr[id];
                    p.push();
                    triangles.drawTriangle([a, b, c], x, y, rot, flip);
                    p.pop();
                }
                
                _drawStaticColors(board);
            }
        };
    };
});

In [ ]:
%%javascript

window.drawStaticBoard = function (name, boardName, element) {
    createSketch(element, name, ['staticBoard'], function (StaticBoard) {
        const [W, H] = [400, 400];

        return function(p) {
            let staticBoard = StaticBoard(p);

            p.setup = function(){
                p.createCanvas(W, H);
                p.noLoop();
            }

            p.draw = function () {
                p.background('#ddd');
                p.push();
                p.translate(W / 2, H / 4);
                staticBoard.drawBoard(boardName);
                p.pop();
            }
        };
    });
}

In [ ]:
send_to_js('staticBoardTest', Board([i, 1] for i in range(N_TRIANGLES)))

In [ ]:
%%javascript

drawStaticBoard('drawTriangles', 'staticBoardTest', element)

### Rotating the triangles

- Encode the rotation as a number in `[0, 1, 2]`
- Use Tween.js for the animation

In [ ]:
%%javascript

require.config({
    paths: {
        tween: 'https://cdnjs.cloudflare.com/ajax/libs/tween.js/17.1.1/Tween.min'
    }
});

In [ ]:
%%javascript

createSketch(element, 'rotate-triangle-demo', ['tween', 'triangles'], function (Tween, Triangles) {
    const W = 300; const H = 150;
    
    return (p) => {
        let obj = { angle: 0 };
        let T = Triangles(p);
        let rotateButton;
        
        let tweenGroup = new Tween.Group();
        let t = new Tween.Tween(obj, tweenGroup)
                    .to({angle: "+" + (p.TWO_PI / 3)}, 500)
                    .easing(Tween.Easing.Quadratic.InOut)
                    .onStart(() => t.running = true)
                    .onComplete(() => t.running = false)
        
        function rotate () {
            if (t.running) return;
            t.start();
        }

        p.setup = function(){
            p.createCanvas(W, H);
            rotateButton = p.createButton('Rotate');
            rotateButton.style('color', '#000');
            rotateButton.mousePressed(rotate);
            rotateButton.position(150, 10);
        }

        p.draw = function () {
            tweenGroup.update();
            p.background('#ddd');
            p.translate(W / 3, H / 2);
            p.push();
            p.rotate(obj.angle);
            T.drawTriangle([0, 1, 2], 0);
            p.pop();
            p.push();
            p.translate(W / 3, 0);
            p.rotate(-obj.angle);
            T.drawTriangle([3, 4, 5], 0);
            p.pop();
        }
    };
});

### Animated Board

In [ ]:
%%javascript

defineModule('animatedBoard',
             ['settings', 'staticBoard', 'triangles', 'tween', 'lodash'],
             (Settings, StaticBoard, Triangles, Tween, _) => {
    
    const {ANIM_W, ANIM_H, N_TRIANGLES} = Settings;
    
    return (p) => {
        let tweenGroup = new Tween.Group();
        let globalTime = 0;
        
        let staticBoard = StaticBoard(p);
        let triangles = Triangles(p);
        
        let [it, animationSpeed, baseSpeed, step] = [0, 1000, 1000, 1];
        let board = null;
        
        // positions around the big triangles (outside)
        const out = _.range(N_TRIANGLES).map(i => {
            return {
                x: ANIM_W * 0.3 + (i % 4) * 100,
                y: Math.floor(i / 4) * 100,
                r: 0,
                f: 0
            };
        });
        
        // positions of the small triangles inside the big triangle
        const pos = triangles.genTrianglesPos().map(t => _.pick(t, ['x', 'y', 'flip']));
        
        // arrays of positions to create the tweens (animations)
        let start = _.range(N_TRIANGLES).map(i => ({x: 0, y: 0, r: 0, f: 0}));
        let end = _.range(N_TRIANGLES).map(i => ({x: 0, y: 0, r: 0, f: 0}));
        
        // store the triangles moving at each turn to display them on top of the others
        let moving = [];
        
        function findPos(triangleId, state) {
            return state.findIndex(e => (e && e[0] === triangleId));
        }

        function transitionState(curr) {
            const states = board.states;
            const [from, to] = [states[curr], states[curr + 1]];
            _.range(N_TRIANGLES).forEach(i => {
                
                const startPos = findPos(i, from);
                const endPos = findPos(i, to);
                
                // one the board
                if (startPos > -1 && endPos > -1) {
                    _.assign(start[i], {x: pos[startPos].x, y: pos[startPos].y, r: from[startPos][1], f: pos[startPos].flip});
                    _.assign(end[i], {x: pos[endPos].x, y: pos[endPos].y, r: to[endPos][1], f: pos[endPos].flip});
                    return;
                }
                
                // not in current state but in the next one
                if (startPos < 0 && endPos > -1) {
                    _.assign(start[i], {x: out[i].x, y: out[i].y, r: out[i].r, f: out[i].f});
                    _.assign(end[i], {x: pos[endPos].x, y: pos[endPos].y, r: to[endPos][1], f: pos[endPos].flip});
                    return;
                }
                
                // in current state but not in the next one, bring back
                if (startPos > -1 && endPos < 0) {
                    _.assign(start[i], {x: pos[startPos].x, y: pos[startPos].y, r: from[startPos][1], f: pos[startPos].flip});
                    _.assign(end[i], {x: out[i].x, y: out[i].y, r: out[i].r, f: out[i].f});
                    return;
                }
                
                // out, no movement
                if (startPos < 0 && endPos < 0) {
                    _.assign(start[i], {x: out[i].x, y: out[i].y, r: out[i].r, f: out[i].f});
                    _.assign(end[i], start[i]);
                    return;
                }
            });

            moving = [];
            start.forEach((a, i) => {
                const b = end[i];
                if (a.x != b.x || a.y != b.y || a.r != b.r) {
                    moving.push(i);
                    new Tween.Tween(a, tweenGroup)
                        .to({x: b.x, y: b.y, r: b.r, f: b.f}, animationSpeed)
                        .easing(Tween.Easing.Quadratic.InOut)
                        .start(globalTime)
                }
            });
            
            new Tween.Tween({t: 0}, tweenGroup)
                .to({t: animationSpeed}, animationSpeed)
                .onComplete(() => {
                    if (it >= states.length - 2) return;
                    updateIterationNumber(it+1);
                    transitionState(curr + Math.min(step, states.length - it - 1));
                })
                .start(globalTime);
            
        }
        
        // controls
        let playButton;
        let running = true;
        let iterationNumber;
        
        function updateIterationNumber(newIt) {
            it = newIt;
            iterationNumber.value(it);
        }
        
        function setState(stateId) {
            tweenGroup.removeAll();
            it = Math.max(0, Math.min(stateId, board.states.length - 2));
            updateIterationNumber(it);
            transitionState(it);
            running = false;
        }

        return {
            init: (boardName) => {
                board = window.kv[boardName];
                playButton = p.createButton('❚❚');
                playButton.style('color', '#000');
                playButton.style('width', '50px');
                playButton.mousePressed(() => running = !running);
                playButton.position(200, 10);
                
                let prevButton = p.createButton('◄');
                prevButton.style('color', '#000');
                prevButton.style('width', '50px');
                prevButton.mousePressed(() => setState(it-1));
                prevButton.position(150, 10);
                
                let nextButton = p.createButton('►');
                nextButton.style('color', '#000');
                nextButton.style('width', '50px');
                nextButton.mousePressed(() => setState(it+1));
                nextButton.position(250, 10);
                
                _.range(3).forEach(i => {
                    let speedRatio = Math.pow(10, i);
                    let speedButton = p.createButton(`x${speedRatio}`);
                    speedButton.position(320 + 50 * i, 10);
                    speedButton.style('width', '50px');
                    speedButton.style('color', '#000');
                    speedButton.mousePressed(() => animationSpeed = baseSpeed / speedRatio);
                });
                
                iterationNumber = p.createInput();
                iterationNumber.position(150, 50);
                iterationNumber.style('width', '50px');
                iterationNumber.style('color', '#000');
                
                let iterationSubmit = p.createButton('Set Iteration');
                iterationSubmit.position(200, 50);
                iterationSubmit.style('width', '100px');
                iterationSubmit.style('color', '#000');
                iterationSubmit.style('font-size', '12px');
                iterationSubmit.mousePressed(() => setState(parseInt(iterationNumber.value(), 10) || 0));
            },
            startAnimation: (startIteration, speed) => {
                it = startIteration;
                baseSpeed = speed;
                animationSpeed = speed;
                transitionState(0);
            },
            draw: () => {
                playButton.html(running ? '❚❚' : '►');
                if (running) {
                    tweenGroup.update(globalTime);  
                    globalTime += Math.min(1000 / p.frameRate(), 33);
                }
                
                p.fill('#000');
                p.textSize(24);
                p.text(`Iteration: ${it}`, 10, 30);
                
                
                p.translate(ANIM_W / 4, ANIM_H / 4);
                
                staticBoard.drawFrame();
                
                let allTriangles = _.range(N_TRIANGLES);
                let staticTriangles = _.difference(allTriangles, moving);
                [staticTriangles, moving].forEach(bucket => {
                    bucket.forEach(triangleId => {
                        const [a, b, c] = board.TRIANGLES[triangleId];
                        const {x, y, r, f} = start[triangleId];
                        p.push();
                        triangles.drawTriangle([a, b, c], x, y, r, f);
                        p.pop();
                    });
                });
                staticBoard.drawStaticColors(board);
            }
        }
    };
});

In [ ]:
%%javascript

window.animateBoard = function (name, boardName, element, startIteration=0, speed=1000) {
    createSketch(element, name, ['animatedBoard', 'settings'], (AnimatedBoard, Settings) => {
        const {ANIM_W, ANIM_H} = Settings;
        
        const W = Math.min(element[0].offsetWidth, 900); 
        const H = W / (ANIM_W / ANIM_H);

        return function(p) {
            let board = AnimatedBoard(p);
            
            p.setup = function () {
                let canvas = p.createCanvas(W, H);
                canvas.style('margin-top', '100px');
                board.init(boardName);
                board.startAnimation(startIteration, speed);
            }

            p.draw = function () {
                p.scale(W / ANIM_W);
                p.background('#ddd');
                board.draw();
            }
        };
    });
}

In [ ]:
board = Board(
    [[i, 0] for i in range(N_TRIANGLES)],
    (
        [None] * 16,
        [[7, 0]] + [None] * 15,
        [[7, 1]] + [None] * 15,
        [[7, 2]] + [None] * 15,
        [[7, 2], [0, 0]] + [None] * 14,
        [[7, 2], [0, 1]] + [None] * 14,
        [[7, 2], [0, 2]] + [None] * 14,
    )
)
send_to_js('testAnimatedBoard', board)

In [ ]:
%%javascript

animateBoard('animatedBoard', 'testAnimatedBoard', element, 0, 2000)

## Recursive Search

In [ ]:
import numpy as np

cumsum = np.cumsum([2 * k + 1 for k in range(4)])

def pack(j, i, c):
    return sum(2 * k + 1 for k in range(j)) + i

def tri(j, i, c):
    n = pack(j, i, c)
    return f'triangles[p[{n}][0]][{c}-p[{n}][1]]'

def unpack(triangle_id):
    j = next(i for i, s in enumerate(cumsum) if s > triangle_id)
    i = 2 * j + 1
    k = max(0, triangle_id - cumsum[max(0, j - 1)] )
    return (j, i, k)

### Iterating over the triangles

![looping triangle](./img/looping_triangle.png)

In [ ]:
def generate_valid_triangle(triangle_id, reverse=False):
    j, i, k = unpack(triangle_id)
    code = []
    if k == 0:
        # on the left edge
        code.append(f'{tri(j, k, 2)} == Board.LEFT[{j}]')

    if k == i - 1:
        # on the right edge
        code.append(f'{tri(j, k, 1)} == Board.RIGHT[{j}]')

    if j == 3 and k % 2 == 0:
        # on the bottom edge
        code.append(f'{tri(j, k, 0)} == Board.BOTTOM[{k//2}]')

    if k % 2 == 0:
        # normal orientation (facing up)
        if j < 3:
            # match with line below
            code.append(f'(not p[{pack(j + 1, k + 1, 0)}] or {tri(j, k, 0)} == {tri(j + 1, k + 1, 0)})')
        if k > 0:
            # match with triangle on the left
            code.append(f'(not p[{pack(j, k - 1, 2)}] or {tri(j, k, 2)} == {tri(j, k - 1, 2)})')
        if k < i - 1:
            # match with triangle on the right
            code.append(f'(not p[{pack(j, k + 1, 1)}] or {tri(j, k, 1)} == {tri(j, k + 1, 1)})')
            
    if k % 2 == 1 and reverse:
        # reverse orientation (facing down)
        # match with line above
        code.append(f'(not p[{pack(j - 1, k - 1, 0)}] or {tri(j, k, 0)} == {tri(j - 1, k - 1, 0)})')
        # match with triangle on the left
        code.append(f'(not p[{pack(j, k - 1, 2)}] or {tri(j, k, 2)} == {tri(j, k - 1, 2)})')
        # match with triangle on the right
        code.append(f'(not p[{pack(j, k + 1, 1)}] or {tri(j, k, 1)} == {tri(j, k + 1, 1)})')
        
    return code

generate_valid_triangle(0, False)

### Valid?

![valid triangle](./img/valid_triangle.png)

In [ ]:
def generate_valid_function():
    code = []
    for n in range(N_TRIANGLES):
        _, _, k = unpack(n)
        if k % 2 != 0:
            continue
        code += [f"(not p[{n}] or ({' and '.join(generate_valid_triangle(n))}))"]
    return 'def is_valid (p):\n\treturn ' + ' and '.join(code) + '\n\t'


code = generate_valid_function()
print('Code:')
print(code)
print('Evaluating the code...')
exec(code)
assert 'is_valid' in globals()
print('done.')

In [ ]:
state = [None for i in range(N_TRIANGLES)]
state[0] = [3, 0]
print(state)
assert not is_valid(state)

state[0] = [0, 2]
print(state)
assert is_valid(state)

In [ ]:
from copy import deepcopy


class RecursiveSolver():
    
    def __init__(self):
        self.reset_state()
        
    def reset_state(self):
        self.board = [None] * N_TRIANGLES
        self.used = [0] * N_TRIANGLES
        self.log = [deepcopy(self.board)]
        self.it = 0
        
    def _log(self):
        self.log.append(deepcopy(self.board))
        
    def _place(self, i):
        self.it += 1
        if i == N_TRIANGLES:
            return True
        
        for j in range(N_TRIANGLES - 1, -1, -1):
            if self.used[j]:
                # piece number j already used
                continue
                
            self.used[j] = 1
            
            for rot in range(3):
                self.board[i] = (j, rot)
                self._log()

                if is_valid(self.board) and self._place(i + 1):
                    return True

            # remove the piece from the board
            self.board[i] = None
            self._log()    
            self.used[j] = 0
            
        return False

    def run(self):
        self.reset_state()
        self._place(0)
        return self.board
    
    def found(self):
        return all(slot is not None for slot in self.board)

In [ ]:
%%time

solver = RecursiveSolver()
res = solver.run()
if solver.found():
    print('Solution found')
    print(res)
    print(f'{len(solver.log)} actions')
    send_to_js('recursive', Board(res, solver.log))
else:
    print('No solution found')

In [ ]:
for s in solver.log[:10]:
    print(s)

In [ ]:
%%javascript

drawStaticBoard('solution-recursive', 'recursive', element)

In [ ]:
%%javascript

animateBoard('animation-recursive', 'recursive', element, 0, 2500)

In [ ]:
from IPython.display import HTML

HTML('<iframe width="900" height="600" src="https://www.youtube.com/embed/lW7mo-9TqEQ?rel=0" frameborder="0" allow="autoplay; encrypted-media" allowfullscreen></iframe>')

![Real Puzzle Solution](./img/bermuda_triangle_solved.jpg)

## Summary

- Combine all the core strengths of Javascript, Python and the Jupyter Notebook
- Core logic in Python
- Rendering in Javascript

## Improvements 

- Transfering data from Python to Javascript (JSON blob)
- Implement a [Jupyter Widget](https://github.com/jupyter-widgets/ipywidgets) instead
- Support for JupyterLab

- Let's create a **pyp5** Python package?  `¯\_(ツ)_/¯`

## References and going further

- [Jake VanderPlas: The Python Visualization Landscape PyCon 2017](https://www.youtube.com/watch?v=FytuB8nFHPQ)
- Drawing from the presentation made by Nicolas P. Rougier: https://github.com/rougier/python-visualization-landscape
- [pythreejs](https://github.com/jovyan/pythreejs): Implemented as an Jupyter Widget
- [bqplot](https://github.com/bloomberg/bqplot): Great library for interactive data exploration
- [matplotlib patches](https://matplotlib.org/gallery/api/patch_collection.html?highlight=circles%20wedges%20polygons): Draw primitive shapes

## Questions?

- **@jtpio** on Twitter
- Source: [https://github.com/jtpio/p5-jupyter-notebook](https://github.com/jtpio/p5-jupyter-notebook)
- [jtp.io](https://jtp.io)